In [20]:
import csv

import numpy as np
from util import (
    cognitive_dissonance_score,
    Strength,
    TOKEN_TO_STRENGTH
)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
firstrow = None
secondrow = None
thirdrow = None
fourthrow = None
with open('carson.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if firstrow is None:
            firstrow = row
        elif secondrow is None:
            secondrow = row
        elif thirdrow is None:
            thirdrow = row
        elif fourthrow is None:
            fourthrow = row
    

self_perception_scores = np.empty(24)
for i, token in enumerate(firstrow[2:]):
    strength_str = token.strip()
    self_perception_scores[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores1 = np.empty(24)
for i, token in enumerate(secondrow[2:]):
    strength_str = token.strip()
    external_perception_scores1[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores2 = np.empty(24)
for i, token in enumerate(thirdrow[2:]):
    strength_str = token.strip()
    external_perception_scores2[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_scores3 = np.empty(24)
for i, token in enumerate(fourthrow[2:]):
    strength_str = token.strip()
    external_perception_scores3[TOKEN_TO_STRENGTH[strength_str].value - 1] = i

external_perception_score = (external_perception_scores1 + external_perception_scores2 + external_perception_scores3)/3.0
# print(self_perception_scores)
# print(external_perception_scores1)
# print(external_perception_scores2)
# print(external_perception_scores3)

cog_diss = cognitive_dissonance_score(external_perception_score, self_perception_scores)
print(cog_diss)

4.279596331234476
